In [66]:
import folium
import pandas as pd
import json
import requests

In [113]:
projects= r'P3_GrantExport.csv'
state_data = pd.read_csv(projects, sep=';')
state_data.set_index(state_data.columns[0], inplace=True)
print(state_data.shape)
state_data = state_data[['University', 'Approved Amount']]
state_data.dropna(subset=['University'], inplace=True)
print(state_data.shape)
state_data = state_data[state_data['University'] != 'Nicht zuteilbar - NA'] #Nicht zuteilbar = NA University
state_data = state_data[state_data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO'] #NPO = is not university
print(state_data)

(63969, 14)
(50988, 2)
                                                          University  \
﻿"Project Number"                                                      
4                                          Université de Genève - GE   
6                                             Universität Basel - BS   
8                                        Université de Fribourg - FR   
9                                        Université de Fribourg - FR   
10                                           Universität Zürich - ZH   
11                                       Université de Lausanne - LA   
13                                         Université de Genève - GE   
14                                       Université de Fribourg - FR   
18                                         Université de Genève - GE   
19                                            Universität Basel - BS   
20                                           Universität Zürich - ZH   
21                                       

In [152]:
def searchCantonGeoNames(university): #returns in which canton the university is in
    name, acronym  = university.split(" - ")
    url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=dmartr'
    payload = {'q': university}
    query = parse.quote(url,safe=':/&=?')
    req = requests.get(url, params=payload)
    req_json = json.loads(req.text)
    if bool(req_json['geonames']):
        return req_json['geonames'][0]['adminCode1'], req_json['geonames'][0]['lat'], req_json['geonames'][0]['lng']
    return "NaN", "NaN", "NaN"
    
def search_cantons(data):
    data['Canton'] = ''
    data['Latitude'] = ''
    data['Longitude'] = ''
    for i in range(len(data)):
        canton, lat, lng = searchCantonGeoNames(data['University'].iloc[i])
        print(data['University'].iloc[i])
        data['Canton'].iloc[i] = canton
        data['Latitude'].iloc[i] = lat
        data['Longitude'].iloc[i] = lng
        print(data['Longitude'].iloc[i])
    data = data[data['Canton'] != 'NaN'] 
    return data

print(search_cantons(state_data))

KeyError: 'geonames'

In [121]:
cantonMap = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 
                                 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR',
                                'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})

ch_geo = r'ch-cantons.topojson.json'

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=cantonMap, 
                columns=['Canton'], 
                geo_path=ch_geo,  
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')
map.save('ch-cantons.html')

IndexError: list index out of range